<a href="https://colab.research.google.com/github/Tensor-Reloaded/PMP-2024/blob/main/Lab03/exemplu_pgmpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [ ]:
# a)
model = DiscreteBayesianNetwork([
  ('O', 'H'),
  ('O', 'W'),
  ('H', 'R'),
  ('W', 'R'),
  ('H', 'E'),
  ('R', 'C'),
])

cpd_O = TabularCPD('O', 2, [[0.3], [0.7]])

cpd_H = TabularCPD('H', 2,
                   [[0.9, 0.2],   # P(H=0 | O)
                    [0.1, 0.8]],  # P(H=1 | O)
                   evidence=['O'], evidence_card=[2])

cpd_W = TabularCPD('W', 2,
                   [[0.1, 0.6],   # P(W=0 | O)
                    [0.9, 0.4]],  # P(W=1 | O)
                   evidence=['O'], evidence_card=[2])

cpd_R = TabularCPD('R', 2,
                   [[0.6, 0.9, 0.3, 0.5],   # P(R=0 | H, W)
                    [0.4, 0.1, 0.7, 0.5]],  # P(R=1 | H, W)
                   evidence=['H', 'W'], evidence_card=[2, 2])

cpd_E = TabularCPD('E', 2,
                   [[0.8], [0.2],
                    [0.2], [0.8]],
                   evidence=['H'], evidence_card=[2])

cpd_C = TabularCPD('C', 2,
                   [[0.85], [0.40],
                    [0.15], [0.60]],
                   evidence=['R'], evidence_card=[2])

model.add_cpds(cpd_C, cpd_E, cpd_R, cpd_W, cpd_H, cpd_O)

assert model.check_model()

In [ ]:

# b)

# infer H
infer = VariableElimination(model)
query_H = infer.query(variables=['H'], evidence={'C': 0})
print(query_H)

# infer E
query_E = infer.query(variables=['E'], evidence={'C': 0})
print(query_E)

# infer H, W
query_H_W = infer.map_query(variables=['H, W'], evidence={'C': 0})
print(query_H_W)

In [ ]:
# c)

print(model.get_independencies())

# W  is independent E | H
# 1. W <- O -> H -> E
# 2. W -> R <- H -> E

# observing H in 1. blocks O -> H -> E making O and E independent
# blocking O -> H -> E also blocks W <- O -> H -> E so W will be independent from E

# observing H in 2. blocks R <- H -> E making R and E independent
# blocking R <- H -> E also blocks W -> R <- H -> E so W will be independent from E

# O is not independent C | R
# observing R in H -> R <- W acctually UNblocks the path C - R - H - O and and C - R - W - O making H and W dependent on each one.
# O is dependent with both H and W so O is not independent C | R.